In [1]:
import os
import tools
import numpy as np
import pandas as pd
import scanpy as sc

sc.logging.print_header()

scanpy==1.8.1 anndata==0.7.6 umap==0.5.1 numpy==1.22.4 scipy==1.6.2 pandas==1.2.4 scikit-learn==0.24.2 statsmodels==0.12.2 python-igraph==0.9.1 pynndescent==0.5.2


In [2]:
os.chdir('/lustre/scratch117/cellgen/team297/kt16/COVID_imperial_renal')

In [3]:
bdata = sc.read_h5ad('h5ad/df.fil3_gex_bcells_no_CLL_vdj_230622.h5ad')
bdata

AnnData object with n_obs × n_vars = 68683 × 330
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_ADT', 'nFeature_ADT', 'nCount_ADT_raw', 'nFeature_ADT_raw', 'nCount_RNA_raw', 'nFeature_RNA_raw', 'soupxOnAdt', 'soupxOnRna', 'percentMito', 'exclude', 'centre', 'pool', 'tcrId', 'bcrId', 'celltypist_broad_predLabel', 'celltypist_broad_maxPredProb', 'haniffa_broad_predLabel', 'haniffa_broad_maxPredProb', 'yoshida_broad_predLabel', 'yoshida_broad_maxPredProb', 'celltypist_detailed_predLabel', 'celltypist_detailed_maxPredProb', 'yoshida_detailed_predLabel', 'yoshida_detailed_maxPredProb', 'seurat_clusters', 'RNA_sequencing_saturation', 'pool_factor', 'RNA_readsConfidentlyOnTranscriptome', 'RNA_readsConfidentlyOnGenome', 'RNA_readsOnGenome', 'RNA_Q30readsBarcode', 'RNA_Q30readsRead', 'RNA_Q30readsUmi', 'rna_snn_res.4', 'rna_snn_res.0.5', 'rna_snn_res.0.1', 'harmony_snn_res.4', 'yoshida_broad_predLabel_majorVote', 'harmony_snn_res.0.5', 'harmony_snn_res.0.1', 'cell_type_compartment

In [4]:
adata = sc.read_h5ad('h5ad/df.fil3_gex_kt_230622.h5ad')
adata = adata[adata.obs_names.isin(bdata.obs_names)]

In [5]:
assert all(adata.obs.index == bdata.obs.index)

In [6]:
adata.obs = bdata.obs.copy()
adata.obs = adata.obs[['orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_ADT', 'nFeature_ADT', 'nCount_ADT_raw', 'nFeature_ADT_raw', 'nCount_RNA_raw', 'nFeature_RNA_raw', 'soupxOnAdt', 'soupxOnRna', 'percentMito', 'exclude', 'centre', 'pool', 'tcrId', 'bcrId', 'celltypist_broad_predLabel', 'celltypist_broad_maxPredProb', 'haniffa_broad_predLabel', 'haniffa_broad_maxPredProb', 'yoshida_broad_predLabel', 'yoshida_broad_maxPredProb', 'celltypist_detailed_predLabel', 'celltypist_detailed_maxPredProb', 'yoshida_detailed_predLabel', 'yoshida_detailed_maxPredProb', 'seurat_clusters', 'RNA_sequencing_saturation', 'pool_factor', 'merged_souporcell_status', 'patient_id', 'sample_id', 'ethnicity', 'sex', 'calc_age', 'ihd', 'previous_vte', 'copd', 'diabetes', 'smoking', 'cause_eskd', 'renal_status', 'WHO_severity', 'admission_date', 'discharge_date', 'fatal_disease', 'case_control', 'date_positive_swab', 'date_first_symptoms', 'radiology_evidence_covid', 'sample_date', 'temp_WCC', 'temp_neut', 'temp_mono', 'temp_lymph', 'temp_CRP', 'temp_ddimer', 'temp_ferritin', 'temp_trop', 'WHO_temp_severity', 'individual_id', 'celltype_B_corrected', 'celltype_B_corrected_IFN', 'corrected_ethnicity', 'PC1_nonafricanVsAfrican', 'PC2_asianVsEuropean']]
adata

AnnData object with n_obs × n_vars = 68683 × 33559
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_ADT', 'nFeature_ADT', 'nCount_ADT_raw', 'nFeature_ADT_raw', 'nCount_RNA_raw', 'nFeature_RNA_raw', 'soupxOnAdt', 'soupxOnRna', 'percentMito', 'exclude', 'centre', 'pool', 'tcrId', 'bcrId', 'celltypist_broad_predLabel', 'celltypist_broad_maxPredProb', 'haniffa_broad_predLabel', 'haniffa_broad_maxPredProb', 'yoshida_broad_predLabel', 'yoshida_broad_maxPredProb', 'celltypist_detailed_predLabel', 'celltypist_detailed_maxPredProb', 'yoshida_detailed_predLabel', 'yoshida_detailed_maxPredProb', 'seurat_clusters', 'RNA_sequencing_saturation', 'pool_factor', 'merged_souporcell_status', 'patient_id', 'sample_id', 'ethnicity', 'sex', 'calc_age', 'ihd', 'previous_vte', 'copd', 'diabetes', 'smoking', 'cause_eskd', 'renal_status', 'WHO_severity', 'admission_date', 'discharge_date', 'fatal_disease', 'case_control', 'date_positive_swab', 'date_first_symptoms', 'radiology_evidence_covid', 'sam

In [7]:
import anndata2ri
anndata2ri.activate()
%load_ext rpy2.ipython

In [8]:
# remove doublets
adata = adata[(adata.obs['merged_souporcell_status'] == 'singlet') & (adata.obs['celltype_B_corrected'] != 'doublet')].copy()
# remove the unannotated samples
adata = adata[(adata.obs['sample_id'] != 'NA') & (~adata.obs['patient_id'].isin(['NULL', 'unresolved'])) & (adata.obs['case_control'] != 'NA')].copy()
# remove patient C141 recovery because C141 was unwell at the time the “recovery sample” was taken (high CRP and hypoxia). They died a few days after.
adata = adata[(adata.obs['sample_id'] != 'C141_0903')].copy()
adata

AnnData object with n_obs × n_vars = 59553 × 33559
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_ADT', 'nFeature_ADT', 'nCount_ADT_raw', 'nFeature_ADT_raw', 'nCount_RNA_raw', 'nFeature_RNA_raw', 'soupxOnAdt', 'soupxOnRna', 'percentMito', 'exclude', 'centre', 'pool', 'tcrId', 'bcrId', 'celltypist_broad_predLabel', 'celltypist_broad_maxPredProb', 'haniffa_broad_predLabel', 'haniffa_broad_maxPredProb', 'yoshida_broad_predLabel', 'yoshida_broad_maxPredProb', 'celltypist_detailed_predLabel', 'celltypist_detailed_maxPredProb', 'yoshida_detailed_predLabel', 'yoshida_detailed_maxPredProb', 'seurat_clusters', 'RNA_sequencing_saturation', 'pool_factor', 'merged_souporcell_status', 'patient_id', 'sample_id', 'ethnicity', 'sex', 'calc_age', 'ihd', 'previous_vte', 'copd', 'diabetes', 'smoking', 'cause_eskd', 'renal_status', 'WHO_severity', 'admission_date', 'discharge_date', 'fatal_disease', 'case_control', 'date_positive_swab', 'date_first_symptoms', 'radiology_evidence_covid', 'sam

In [9]:
# calculate days to sampling, days to admission, days to first symptoms, days to discharge from positive swab
tmp = adata.copy()
tmp.obs['sample_date'] = tmp.obs['sample_date'].astype('object')
tmp.obs['date_first_symptoms'] = tmp.obs['date_first_symptoms'].astype('object')
tmp.obs['date_positive_swab'] = tmp.obs['date_positive_swab'].astype('object')
tmp.obs['date_first_symptoms'].replace({'NA':np.nan}, inplace = True)
tmp.obs['date_positive_swab'].replace({'NA':np.nan}, inplace = True)
time_from_first_symptoms = pd.to_datetime(tmp.obs['sample_date'], format='%d/%m/%Y') - pd.to_datetime(tmp.obs['date_first_symptoms'], format='%d/%m/%Y')
time_from_positive_swab = pd.to_datetime(tmp.obs['sample_date'], format='%d/%m/%Y') - pd.to_datetime(tmp.obs['date_positive_swab'], format='%d/%m/%Y')
days_from_first_symptoms = time_from_first_symptoms.dt.days
days_from_positive_swab = time_from_positive_swab.dt.days
days_from_first_symptoms.fillna(0, inplace = True)
days_from_positive_swab.fillna(0, inplace = True)

/nfs/users/nfs_k/kt16/miniconda2/envs/dandelion/lib/python3.8/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: modifications to a property of a datetimelike object are not supported and are discarded. Change values on the original.
  return super().fillna(


In [10]:
min(days_from_first_symptoms)

-9.0

In [11]:
min(days_from_positive_swab)

0.0

In [12]:
days_from_infection = [max(a,b) for a,b in zip(days_from_first_symptoms, days_from_positive_swab)]

In [13]:
min(days_from_infection)

0.0

In [14]:
adata.obs['time_from_first_symptoms'] = days_from_first_symptoms
adata.obs['time_from_positive_swab'] = days_from_positive_swab
adata.obs['time_from_infection'] = days_from_infection

### replace counts

In [15]:
adata.X = adata.layers['counts'].copy()
adata

AnnData object with n_obs × n_vars = 59553 × 33559
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_ADT', 'nFeature_ADT', 'nCount_ADT_raw', 'nFeature_ADT_raw', 'nCount_RNA_raw', 'nFeature_RNA_raw', 'soupxOnAdt', 'soupxOnRna', 'percentMito', 'exclude', 'centre', 'pool', 'tcrId', 'bcrId', 'celltypist_broad_predLabel', 'celltypist_broad_maxPredProb', 'haniffa_broad_predLabel', 'haniffa_broad_maxPredProb', 'yoshida_broad_predLabel', 'yoshida_broad_maxPredProb', 'celltypist_detailed_predLabel', 'celltypist_detailed_maxPredProb', 'yoshida_detailed_predLabel', 'yoshida_detailed_maxPredProb', 'seurat_clusters', 'RNA_sequencing_saturation', 'pool_factor', 'merged_souporcell_status', 'patient_id', 'sample_id', 'ethnicity', 'sex', 'calc_age', 'ihd', 'previous_vte', 'copd', 'diabetes', 'smoking', 'cause_eskd', 'renal_status', 'WHO_severity', 'admission_date', 'discharge_date', 'fatal_disease', 'case_control', 'date_positive_swab', 'date_first_symptoms', 'radiology_evidence_covid', 'sam

### remove the viral and VDJ features

In [16]:
import re
adata.var['GEX'] = [False if re.search('VIRAL|^IG[HKL][VDJC]|^TR[ABDG][VDJ]', i) else True for i in adata.var.index]
adata = adata[:,adata.var.GEX]
adata

View of AnnData object with n_obs × n_vars = 59553 × 32913
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_ADT', 'nFeature_ADT', 'nCount_ADT_raw', 'nFeature_ADT_raw', 'nCount_RNA_raw', 'nFeature_RNA_raw', 'soupxOnAdt', 'soupxOnRna', 'percentMito', 'exclude', 'centre', 'pool', 'tcrId', 'bcrId', 'celltypist_broad_predLabel', 'celltypist_broad_maxPredProb', 'haniffa_broad_predLabel', 'haniffa_broad_maxPredProb', 'yoshida_broad_predLabel', 'yoshida_broad_maxPredProb', 'celltypist_detailed_predLabel', 'celltypist_detailed_maxPredProb', 'yoshida_detailed_predLabel', 'yoshida_detailed_maxPredProb', 'seurat_clusters', 'RNA_sequencing_saturation', 'pool_factor', 'merged_souporcell_status', 'patient_id', 'sample_id', 'ethnicity', 'sex', 'calc_age', 'ihd', 'previous_vte', 'copd', 'diabetes', 'smoking', 'cause_eskd', 'renal_status', 'WHO_severity', 'admission_date', 'discharge_date', 'fatal_disease', 'case_control', 'date_positive_swab', 'date_first_symptoms', 'radiology_evidence_covi

### Split this object to each celltype of interest. 

In [17]:
adatas = {}

for x in sorted(list(set(adata.obs['celltype_B_corrected']))):
    adatas[x] = adata[adata.obs['celltype_B_corrected'] == x]

In [18]:
# check the size
for x in adatas:
    print(x)
    print(adatas[x].n_obs)

B_ASC_IgA
2862
B_ASC_IgG
1460
B_ASC_IgM
564
B_ASC_dividing
1516
B_ASC_other
38
B_CD11c
1889
B_naive
38412
B_non-switched_mem
3316
B_switched_mem
9496


In [19]:
B_ASC_IgG = adatas['B_ASC_IgG']
B_ASC_IgM = adatas['B_ASC_IgM']
B_ASC_IgA = adatas['B_ASC_IgA']
B_ASC_dividing = adatas['B_ASC_dividing']
B_switched_mem = adatas['B_switched_mem']
B_nonswitched_mem = adatas['B_non-switched_mem']
B_naive = adatas['B_naive']
B_CD11c = adatas['B_CD11c']

In [20]:
# group all ASCs together
B_ASC = adata[adata.obs['celltype_B_corrected'].isin(['B_ASC_IgG', 'B_ASC_IgM', 'B_ASC_IgA', 'B_ASC_other'])]
# group all mems together
B_mem = adata[adata.obs['celltype_B_corrected'].isin(['B_switched_mem', 'B_non-switched_mem'])]
# group switched ASCs together
B_ASC_s = adata[adata.obs['celltype_B_corrected'].isin(['B_ASC_IgG', 'B_ASC_IgA'])]

In [21]:
%%R -i B_ASC_IgG
saveRDS(B_ASC_IgG, 'h5ad/df.fil3_gex_bcells_vdj_sce_B_ASC_IgG_230622.RDS')

In [22]:
%%R -i B_ASC_IgA
saveRDS(B_ASC_IgA, 'h5ad/df.fil3_gex_bcells_vdj_sce_B_ASC_IgA_230622.RDS')

In [23]:
%%R -i B_ASC_IgM
saveRDS(B_ASC_IgM, 'h5ad/df.fil3_gex_bcells_vdj_sce_B_ASC_IgM_230622.RDS')

In [24]:
%%R -i B_ASC_dividing
saveRDS(B_ASC_dividing, 'h5ad/df.fil3_gex_bcells_vdj_sce_B_ASC_dividing_230622.RDS')

In [25]:
%%R -i B_switched_mem
saveRDS(B_switched_mem, 'h5ad/df.fil3_gex_bcells_vdj_sce_B_switched_mem_230622.RDS')

In [26]:
%%R -i B_nonswitched_mem
saveRDS(B_nonswitched_mem, 'h5ad/df.fil3_gex_bcells_vdj_sce_B_nonswitched_mem_230622.RDS')

In [27]:
%%R -i B_naive
saveRDS(B_naive, 'h5ad/df.fil3_gex_bcells_vdj_sce_B_naive_230622.RDS')

In [28]:
%%R -i B_CD11c
saveRDS(B_CD11c, 'h5ad/df.fil3_gex_bcells_vdj_sce_B_CD11c_230622.RDS')

In [29]:
%%R -i B_ASC
saveRDS(B_ASC, 'h5ad/df.fil3_gex_bcells_vdj_sce_B_ASC_all_230622.RDS')

In [30]:
%%R -i B_mem
saveRDS(B_mem, 'h5ad/df.fil3_gex_bcells_vdj_sce_B_mem_all_230622.RDS')

In [31]:
%%R -i B_ASC_s
saveRDS(B_ASC_s, 'h5ad/df.fil3_gex_bcells_vdj_sce_B_ASC_switched_230622.RDS')